In [117]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, Row
import pydeequ, pyspark
from pyspark.sql.types import *
from pydeequ import Check,CheckLevel
from pydeequ.checks import *
from pydeequ.verification import *
from pydeequ.analyzers import *
from pyspark.sql.functions import lit,current_timestamp
from configparser import ConfigParser
import sys
import calendar,time

In [33]:
Parser = ConfigParser()
configFile = r"C:\Users\Shivam\Desktop\training\godaddy\godaddy\config\config_analysis.json"
Parser.read(configFile)
#Env = "merchants"
#SourceFileName = "merchants.csv"

##Set Job Parameters
#Raw_S3_Path = Parser.get(Env, 'raw_s3_path')
#Clean_S3_Path = Parser.get(Env, 'clean_s3_path')
#Format = Parser.get(Env, 'file_format')
#print("S3_Path = " + Raw_S3_Path)
#print("Format = " + Format)
#print("SourceFile = " + S3_Path )

['C:\\Users\\Shivam\\Desktop\\training\\godaddy\\godaddy\\config\\config_analysis.json']

In [36]:
list(Parser.keys())

['DEFAULT', 'merchants', 'users']

In [35]:
list(Parser['merchants'])

['data_source', 'raw_s3_path', 'file_format', 'checks']

In [39]:
def allAnalyzers(source_table):
    analyzers = Parser.get(source_table,'checks').split(',')
    return analyzers

In [40]:
list(map(allAnalyzers,list(Parser.keys())[1:]))

[["Completeness('id')",
  " Distinctness('id')",
  " Minimum('id')",
  " Maximum('id')",
  " Completeness('telephone')"],
 ["Completeness('id')",
  " Completeness('email')",
  " Minimum('sign_in_count')",
  " Maximum('sign_in_count')"]]

In [41]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.jars", "C:\Program Files (x86)\PostgreSQL\pgJDBC\postgresql-42.2.18.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)\
    .getOrCreate()

In [44]:
df_merchants = spark.read.format('csv').load(Parser.get('merchants','raw_s3_path'),header=True,inferSchema=True).select('id','state','updated_at','telephone')
df_users = spark.read.format('csv').load(Parser.get('users','raw_s3_path'),header=True,inferSchema=True)

In [149]:
'''analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(eval("Size()"))

res = lambda fun : analysisResult.addAnalyzer(eval(fun))

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, list(map(res,analyzers))[0].run())
analysisResult_df.show() 
'''

'analysisResult = AnalysisRunner(spark)                     .onData(df)                     .addAnalyzer(eval("Size()"))\n\nres = lambda fun : analysisResult.addAnalyzer(eval(fun))\n\nanalysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, list(map(res,analyzers))[0].run())\nanalysisResult_df.show() \n'

In [147]:
def Result(analyzers,df, source_table):
    analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(eval("Size()"))

    res = lambda fun : analysisResult.addAnalyzer(eval(fun))

    analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, list(map(res,analyzers))[0].run())
    analysisResult_df = analysisResult_df.withColumn('ingestion_tsp',lit(current_timestamp()))
    analysisResult_df = analysisResult_df.withColumn('source_table',lit(source_table))
    analysisResult_df.write.csv(f'..//analyzer_files//{source_table}_{calendar.timegm(time.gmtime())}',header=True,mode='overwrite')

    analysisResult_df.show()

In [150]:
'''list_analyzers = list(map(allAnalyzers,list(Parser.keys())[1:]))
#list(map(Result,list_analyzers,['df_merchants','df_users']))
Result(list_analyzers[1],df_users)
'''

"list_analyzers = list(map(allAnalyzers,list(Parser.keys())[1:]))\n#list(map(Result,list_analyzers,['df_merchants','df_users']))\nResult(list_analyzers[1],df_users)\n"

In [148]:
print(*map(Result,list_analyzers,[df_merchants,df_users],list(Parser.keys())[1:]))

+-------+---------+------------+--------+--------------------+------------+
| entity| instance|        name|   value|       ingestion_tsp|source_table|
+-------+---------+------------+--------+--------------------+------------+
|Dataset|        *|        Size|  1000.0|2020-12-22 22:55:...|   merchants|
| Column|       id|Distinctness|     1.0|2020-12-22 22:55:...|   merchants|
| Column|       id|Completeness|     1.0|2020-12-22 22:55:...|   merchants|
| Column|       id|     Minimum|   469.0|2020-12-22 22:55:...|   merchants|
| Column|       id|     Maximum|174577.0|2020-12-22 22:55:...|   merchants|
| Column|telephone|Completeness|   0.999|2020-12-22 22:55:...|   merchants|
+-------+---------+------------+--------+--------------------+------------+

+-------+-------------+------------+---------+--------------------+------------+
| entity|     instance|        name|    value|       ingestion_tsp|source_table|
+-------+-------------+------------+---------+--------------------+----------

In [ ]:
#analysisResult = AnalysisRunner(spark) \
 #                   .onData(df) \
  #                  .addAnalyzer(eval("Size()"))

#for fun in analyzers:
 #   analysisResult = analysisResult.addAnalyzer(eval(fun))
#analysisResult = analysisResult.run()
  #.addAnalyzer(CountDistinct("state")) \
                  
                   

#analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
#analysisResult_df.show()